In [5]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(1, "../")

import SIR_utils as sir
from get_mask_dates import *

In [6]:
mandate = pd.read_csv('../stay_at_home_and_masks.csv')
def get_date(dataframe, state):
    state = dataframe.loc[dataframe['state'] == state]
    date = np.array(state['mask_date'])[0].split('-')[::-1]
    return date[0] + '-' + date[1] + '-' + date[2]

#print(mandate)

In [7]:
# state = "new-jersey"
# state_abbr = "NJ"
state = "florida"
state_abbr = "FL"
# state = "georgia"
# state_abbr = "GA"
# state = "new-york"
# state_abbr = "NY"

time_dataframe = pd.read_csv("../{}-history.csv".format(state))
#print(time_dataframe)

dates = time_dataframe["date"][::-1]
removed = time_dataframe["positiveCasesViral"][::-1]
indices = np.array(1 - np.isnan(removed)).astype(np.bool)

dates = np.array(dates)[indices]
rem = np.array(removed[indices])

#print(dates)
#print(rem)

<ipython-input-7-7deeb894df69>:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = np.array(1 - np.isnan(removed)).astype(np.bool)


In [8]:
window = 14
infectious, infectious_rate = sir.compute_number_infectious(rem, window)

rem = rem[:len(infectious)]

population = np.array(mandate[mandate['state'] == state_abbr]['Population']) 
sus = population - infectious - rem
#print(sus)

192


In [9]:
.row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', 
                             item="stay_at_home_start")
bools = time_dataframe["date"] == np.array(row["date"])[0]
index_home = len(bools) - np.arange(len(bools))[bools]

try:
    row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', 
                                 item="stay_at_home_end")
    bools = time_dataframe["date"] == np.array(row["date"])[0]
    index_out = len(bools) - np.arange(len(bools))[bools]
except:
    print("no stay at home end")

try:
    row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', )
    bools = time_dataframe["date"] == np.array(row["date"])[0]
    index_mask = len(bools) - np.arange(len(bools))[bools]
except:
    index_mask = None

FileNotFoundError: [Errno 2] No such file or directory: 'stay_at_home_and_masks.csv'

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(times, infectious, "*")
plt.vlines(index_home[0], 0, np.max(infectious), "g", label="stay at home order")
try:
    plt.vlines(index_mask[0], 0, np.max(infectious), "r", label="mask mandate")
except:
    print("no mask mandate")

try:
    plt.vlines(index_out[0], 0, np.max(infectious), "y", label="stay at home ends")
except:
    print("no out time")
plt.xlabel("time")
plt.ylabel("infectious")
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
plt.scatter(infectious, infectious_rate, c=times)
plt.plot(infectious[index_home[0]], infectious_rate[index_home[0]], "r^",  
           markersize=20)
plt.plot(infectious[index_out[0]], infectious_rate[index_out[0]], "r^",  
           markersize=20)
plt.ylabel("di/dt")
plt.ylabel("di/dt")
plt.xlabel("infectious")
plt.show()



In [ ]:
times = np.arange(len(rem))
sus_fit = sus / population
inf_fit = infectious / population
rem_fit = rem / population
sir_fitting = sir.moving_averages_fits(times, sus_fit, inf_fit, rem_fit, window=window, 
                                     opt_options={"method":"trust-constr", 
                                                                           "bounds": 
                                                                            [(1e-10, 100), (0, np.inf)]})

In [ ]:
print(sir_fitting)

In [ ]:
plt.figure(figsize=(10, 10))
for j in range(int(len(times) / window)):
    start = j * window
    stop = start + window
    plt.plot(times[start:stop], rem_fit[start:stop], "b*")
    sol = solve_ivp(sir.sir_ode, [times[start], times[stop]], 
                   [sus_fit[start], inf_fit[start], rem_fit[start]], 
                   args=sir_fitting[j, :], method="LSODA", max_step=0.25)
    plt.plot(sol.t, sol.y[2, :], "k--")
    
plt.xlabel("time")
plt.ylabel("removed")
plt.show()

In [ ]:
row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', 
                             item="stay_at_home_start")
bools = time_dataframe["date"] == np.array(row["date"])[0]
index_home = len(bools) - np.arange(len(bools))[bools]

try:
    row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', 
                                 item="stay_at_home_end")
    bools = time_dataframe["date"] == np.array(row["date"])[0]
    index_out = len(bools) - np.arange(len(bools))[bools]
except:
    print("no stay at home end")

try:
    row = mask_date_to_all_states(state_abbr, "all-states-history.csv", 'stay_at_home_and_masks.csv', )
    bools = time_dataframe["date"] == np.array(row["date"])[0]
    index_mask = len(bools) - np.arange(len(bools))[bools]
except:
    index_mask = None

In [ ]:
sir_fitting_plot = np.zeros([len(times), 2])
for j in range(int(len(times) / window)):
    sir_fitting_plot[j * window: j * window + window, :] = sir_fitting[j, :]
    
plt.figure(figsize=(8,6))
plt.plot(times, sir_fitting_plot[:, 0])
plt.vlines(index_home[0], 0, np.max(sir_fitting[:, 0]), "g", label="stay at home order")
try:
    plt.vlines(index_mask[0], 0, np.max(sir_fitting[:, 0]), "r", label="mask mandate")
except:
    print("no mask mandate")

try:
    plt.vlines(index_out[0], 0, np.max(sir_fitting[:, 0]), "y", label="stay at home ends")
except:
    print("no out time")
plt.xlabel("time")
plt.ylabel("viralence")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
plt.plot(times, sir_fitting_plot[:, 1])
plt.vlines(index_home[0], 0, np.max(sir_fitting[:, 1]), "g", label="stay at home order")
try:
    plt.vlines(index_mask[0], 0, np.max(sir_fitting[:, 1]), "r", label="mask mandate")
except:
    print("no mask mandate")

try:
    plt.vlines(index_out[0], 0, np.max(sir_fitting[:, 1]), "y", label="stay at home ends")
except:
    print("no out time")
plt.xlabel("time")
plt.ylabel("removal rate")
plt.legend()
plt.show()